<a href="https://colab.research.google.com/github/Kotya2001/HotelID/blob/ViewData/KaggleTaskAnalitics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Data/train.csv')

In [4]:
print(f'Number of records {len(df)}')
print(f'Number of hotels {df.hotel_id.nunique()}')
print(f'Number of chains {df.chain.nunique()}')
print(f'Number of images {df.image.nunique()}')

Number of records 97556
Number of hotels 7770
Number of chains 88
Number of images 97554


In [5]:
df[df.image.duplicated(keep=False)]

,image,chain,hotel_id,timestamp
56845,c7c7383898d76598.jpg,6,60918,2020-07-18 00:03:38
56846,c7c7383898d76598.jpg,6,60918,2020-08-29 04:35:13
62238,ce27d36c9147cc19.jpg,6,60918,2020-07-18 00:03:38
62239,ce27d36c9147cc19.jpg,6,60918,2020-08-29 04:35:13


In [6]:
df = df.drop_duplicates(subset=['image'], keep='first')

In [7]:
df

,image,chain,hotel_id,timestamp
0,8000d59f6a3d765f.jpg,85,24607,2018-11-28 05:42:31
1,8000febe81817f7f.jpg,6,25389,2017-01-01 07:01:46
2,80053befc2d42fba.jpg,3,672,2018-01-26 13:22:07
3,8007fa0a8fe89f9d.jpg,5,63866,2020-07-03 02:31:31
4,800a4a6b6f6d2df6.jpg,0,12036,2017-09-22 17:59:22
...,...,...,...,...
97551,ffff8082ec219892.jpg,83,20544,2016-12-18 03:04:54
97552,ffff8200e50cc0f1.jpg,82,51053,2018-01-23 03:51:42
97553,ffff84129981a4a4.jpg,83,42842,2017-03-30 22:45:24
97554,ffff85015040c2af.jpg,82,6261,2016-11-09 12:34:06


In [8]:
chain_group_df = df.groupby(["chain"]).agg({"hotel_id": [pd.Series.nunique], "image" : [pd.Series.nunique]})

In [9]:
chain_group_df.columns = ["_".join(x) for x in chain_group_df.columns.ravel()]
chain_group_df = chain_group_df.reset_index().sort_values("hotel_id_nunique")[::-1]

In [10]:
chain_group_df

,chain,hotel_id_nunique,image_nunique
0,0,1749,20259
6,6,698,8157
5,5,609,6328
86,90,270,3318
3,3,266,5398
...,...,...,...
12,12,1,26
16,18,1,8
10,10,1,12
52,56,1,8


In [33]:
fig = px.scatter(chain_group_df, x="chain", y="hotel_id_nunique",
                 size="image_nunique", color = "image_nunique",
                 hover_name = None,
                 log_y=True, size_max=75)

fig.update_yaxes(title_text="Hotel count")
fig.update_xaxes(title_text="Chain ID")
fig.update_layout(title="Hotel and image count per chain", coloraxis=dict(colorbar=dict(title="Image count")))
fig.update_traces(hovertemplate="Chain: %{x} <br>Hotel count: %{y}<br>Image count: %{marker.size}")
fig.show()

In [30]:
grouped_df = df.groupby(['chain', 'hotel_id']).size().to_frame('image_count').sort_values("image_count")[::-1].reset_index()

In [32]:
fig = px.histogram(grouped_df, x="image_count", nbins=25, marginal="box", height=500)
fig.update_layout(title="Distribution of image count per hotel")
fig.update_traces(hovertemplate="Image count: %{x} <br>Hotel count: %{y}")
fig.show()